#### Problem Statement

1) Describe the data set and eventual anomalies you find.    
2) Which patterns do you find in the purchasing behavior of the customer?            
3) What are the categories and genres which customers are mostly intrested in?     
4) Split the customers in different groups based on their purchasing behavior.     
   a) Justify your choice for adopted method(s) and model(s).                          
   b) Describe the defined customer groups. What are the features which are driving the differentiation amongst the different groups?    
   c) Give suggestions how the business should treat these clusters differently.        
5) Assuming that the 'Category_Reporting' tells you the category of all the items in that order, predic:    
   a) The number of items per category which will be ordered on a monthly basis for rest of May 2021.    
   b) The number of returns for the rest of May 2021.      
6) As, at this point in your analysis you are the dataset expert, suggest any ideas (initiatives, further analysis) you might have in mind which can be helpful for the business.    
   

In [17]:
# loading packages
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import math
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

In [15]:
# reading the data set
df = pd.read_csv(r"C:\Users\Owner\OneDrive\Desktop\Python Code\EMP Systems\EMP_DataScientist_CS\Data\EMPCaseStudy_v1.csv")

In [18]:
df.head(3)

,CustomerHash,OrderHash,Customer_Age,Gender,Account_Age,AffinityProductGender,Category_Reporting,Genre,ClientType,PartnerProgram1,PartnerProgram2,FirstOrder,Marketing_Channel,Pieces_Ordered,Pieces_Outbound,Pieces_Returns,Pieces_Fulfilled,OrderValue,Revenue_Goods,Delivery_Value,Return_Value,Discount_Total,OrderProfit,DateTimeOrder
0,0xA09E6101DF2E146E55BA9183F8E3994D98F5C963,0x8B9731CAF4EDB3333DC0B29D92399C654692BFE3,53.0,Man,10,Female,Fashion / Private label - Apparel,Private label,desktop,No,No,0,direct,1,1.0,NaN,1.0,53.2185,53.22,53.22,NaN,-5.5966,32.8376,2019-11-29
1,0x678B7B262CE7FFE0EEC0AD8FD784AA1F46C65875,0xBFD162E330E8783658BB4F2A26B05A300FEC6253,NaN,Woman,0,Unisex,Warner D2C,Warner D2C,mobile,No,No,1,seo,1,1.0,NaN,1.0,29.4034,29.40,29.40,NaN,-12.6100,17.0572,2019-11-23
2,0xEDC6F42832CD97571C60E48212855E142D2309C4,0x5513669E48EAEB800E965C9428531EBCAE3EC20A,NaN,Man,0,Male,Fashion / Private label - Apparel,Private label,mobile,No,No,0,newsletter,1,1.0,NaN,1.0,33.6050,33.61,33.61,NaN,0.0000,17.1399,2019-11-28


In [19]:
# printing shape of provided data set
print("Print shape of data set: ",df.shape)

Print shape of data set:  (1119023, 24)


In [20]:
# finding total number of duplicate values in data set if any
print('Total number of duplicate values in the data set is/are: {}'.format(df.duplicated().sum()))

Total number of duplicate values in the data set is/are: 0


In [21]:
# checking for null values in the data set
col = df.columns
for i in col:
    # count number of rows with missing values
    n_miss = df[[i]].isnull().sum()
    perc = n_miss / df.shape[0] * 100
    print('%s, Missing: %d (%.1f%%)' % (i, n_miss, perc))

CustomerHash, Missing: 0 (0.0%)
OrderHash, Missing: 0 (0.0%)
Customer_Age, Missing: 327211 (29.2%)
Gender, Missing: 0 (0.0%)
Account_Age, Missing: 0 (0.0%)
AffinityProductGender, Missing: 0 (0.0%)
Category_Reporting, Missing: 0 (0.0%)
Genre, Missing: 0 (0.0%)
ClientType, Missing: 11606 (1.0%)
PartnerProgram1, Missing: 0 (0.0%)
PartnerProgram2, Missing: 0 (0.0%)
FirstOrder, Missing: 0 (0.0%)
Marketing_Channel, Missing: 3 (0.0%)
Pieces_Ordered, Missing: 0 (0.0%)
Pieces_Outbound, Missing: 22253 (2.0%)
Pieces_Returns, Missing: 893123 (79.8%)
Pieces_Fulfilled, Missing: 22253 (2.0%)
OrderValue, Missing: 0 (0.0%)
Revenue_Goods, Missing: 22253 (2.0%)
Delivery_Value, Missing: 22253 (2.0%)
Return_Value, Missing: 893123 (79.8%)
Discount_Total, Missing: 0 (0.0%)
OrderProfit, Missing: 22253 (2.0%)
DateTimeOrder, Missing: 0 (0.0%)


In [26]:
# Checking the rows where Pieces_Outbound is missing. This row represents the total number of items shipped to the customer.
df[df["Pieces_Outbound"].isnull()]

,CustomerHash,OrderHash,Customer_Age,Gender,Account_Age,AffinityProductGender,Category_Reporting,Genre,ClientType,PartnerProgram1,PartnerProgram2,FirstOrder,Marketing_Channel,Pieces_Ordered,Pieces_Outbound,Pieces_Returns,Pieces_Fulfilled,OrderValue,Revenue_Goods,Delivery_Value,Return_Value,Discount_Total,OrderProfit,DateTimeOrder
155,0xF73BB08DB3F4AF9E5B561F3CD695DB3DB0A27B60,0x26BDC5573C9DD368BB817340C24E00C454E26EBA,50.0,Woman,0,Female,Music - Apparel,Merchandise - Music,mobile,No,No,0,sea nonbrand,2,NaN,NaN,NaN,37.7983,NaN,NaN,NaN,0.0000,NaN,2019-11-29
178,0x093C49137543113AFAD99E83B49FE8FCE4FE4715,0x32EEDEAF1C25F026908D20BA6F9149C557F717C3,42.0,Man,4,Male,Fashion - Accessories,Merchandise - Fun,NaN,No,No,0,unknown,1,NaN,NaN,NaN,25.2017,NaN,NaN,NaN,0.0000,NaN,2019-11-30
210,0x881B13322EDAD5AE43BBC9EBF4701AAA41CAC694,0xFB232D5DF949EAA9EC9346E757E801DF82D197B5,46.0,Woman,0,Female,Fashion / Private label - Apparel,Private label,desktop,No,No,0,retargeting,2,NaN,NaN,NaN,33.5966,NaN,NaN,NaN,0.0000,NaN,2019-11-17
239,0x18A140FE14856C118697FC0FF3DA13100A2909FD,0xD9D5BB566C60AA0C25569BADBB5E6914119721DC,44.0,Woman,9,Female,Fashion - Accessories,3rd Party Brands,app,No,No,0,direct,2,NaN,NaN,NaN,20.1513,NaN,NaN,NaN,0.0000,NaN,2019-11-26
335,0x006262D72C5E1A066CB0BD2C4F30F50815BD3DA6,0x92590690A51F928774B5F9883F6C53273824E5CD,43.0,Undefined,9,Unisex,Fashion / 3rd Party - Apparel,3rd Party Brands,NaN,No,No,0,unknown,2,NaN,NaN,NaN,100.7479,NaN,NaN,NaN,0.0000,NaN,2019-11-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1118965,0x5AE7AF668BC8A9AF1B3DF6010EA5668BBA5B000C,0x8A0814A73DE78FE6DFDE689E16C0425FF51CD519,44.0,Woman,2,Female,Music - Apparel,Merchandise - Music,mobile,Yes,No,0,sea brand,1,NaN,NaN,NaN,49.0168,NaN,NaN,NaN,-5.5900,NaN,2021-05-02
1118977,0xF52F035F1FB4AD3007AC76A70E62B4EB0A5C34BE,0x3A190D010D24B90A271D0AD20D323994F21FF717,20.0,Man,4,Male,Media,Merchandise - Music,mobile,Yes,No,0,social media,1,NaN,NaN,NaN,54.6134,NaN,NaN,NaN,0.0000,NaN,2021-05-03
1118985,0x692815F2D27531597D9B43F183D962ED8D6F3B19,0x7996E4708E552399BB173731EE8A8B53C12A493B,54.0,Man,15,Unisex,Media,Merchandise - Music,mobile,Yes,Yes,0,direct,3,NaN,NaN,NaN,57.1177,NaN,NaN,NaN,0.0000,NaN,2021-05-09
1118994,0x2AC4C2E1B747625A564AC286F56BC63F32D227FA,0x1FB0E8D0BD1B89C068B515C19DAE93CE0CBFE142,32.0,Man,0,Unisex,Warner D2C,Warner D2C,desktop,No,No,0,direct,1,NaN,NaN,NaN,50.4118,NaN,NaN,NaN,0.0000,NaN,2021-05-10


From the data set we can see that their is no duplicate values but their are few missing values which we have to handle before applying our ML algorithm.   
1) We can see that almost 80% of data is missing in Pieces_Returns and simmillarly, 80% in Return_Value.   
2) There is 29.2% data is missing in customer age column to impute these values first we will use KNNImputer to impute these values.    
3) ClientType 1% of data is missing may be we can drop the rows where the client type is missing since we have big data set.   
4) We have 2% missing data in Pieces_Outbound.    
5) We have 2% missing data in Pieces_Fulfilled.    
6) We have 2% missing data in Revenue_Goods.   
7) We have 2% missing data in Delivery_Value.    
8) We have 2% missing data in OrderProfit.    
##### Assumption:    
1) I assume that for missing values in these fields mean that no item has been returned so if no item is returned than no return value. In this case I will mark all the missing values with zero.     